# Python Bellabeat Case Study
Last updated: 08/08/2021

Bellabeat is a company that focuses on creating smart products that focus on the health and wellbeing of women.

## The Business Task

One of the founders, Urska Srsen has tasked us with analyzing smart device usage to understand how consumers use non-Bellabeat devices and apply those insights to one Bellabeat product. We will focus on answering three main questions: What are some trends in smart device usage? How could these trends apply to Bellabeat customers? How could these trends help influence Bellabeat marketing strategy?

# The Data

The [data](https://www.kaggle.com/arashnic/fitbit) for this analysis comes from a dataset uploaded by the user arashnic on Kaggle called FitBit Fitness Tracker Data.  They collected this data from 33 respondents to a survey on Amazon Mechanical Turk from 3/12/2016 to 5/12/2016 and contains entries from 4/12/2016 to 5/12/206.  There is one main file that contains the daily number of steps, calories, minutes of activity for each level, and the distance travelled.  Additional files breakdown the daily values by hour and minute, and others contain the users heartrate every 5 to 30 seconds, the amount of sleep they got each day, and the weight of some of the users.  The file containing the weight information only has data from 8 users, so this file will not be used as it is not enough data to perform an accurate analysis. Additionally, there is no further information on the people who submitted the data such as their age or gender.

# Cleaning nad Organizing the Data

First step is to import the data.

In [ ]:
#import relevant libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

daily_activity = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
daily_sleep = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")
heartrate_seconds = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv")
minute_steps = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/minuteStepsNarrow_merged.csv")
minute_mets = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/minuteMETsNarrow_merged.csv")
minute_calories = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/minuteCaloriesNarrow_merged.csv")
hourly_intensities = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlyIntensities_merged.csv")
hourly_steps = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv")

daily_activity.info()
daily_sleep.info()
heartrate_seconds.info()
minute_steps.info()
minute_mets.info()
minute_calories.info()
hourly_intensities.info()
hourly_steps.info()

A quick query in BigQuery:

SELECT Id, COUNT(ActivityDate) AS total_days

FROM dailyActivity_merged.csv

GROUP BY Id

shows that not all users have 30 days of data. However, there is still enough data to perform an analysis.

This can be recreated in Python as well.

In [ ]:
print(daily_activity.groupby("Id")["ActivityDate"].count())

We can clearly see there is a user with only four entries. We can remove this user.

In [ ]:
daily_activity = daily_activity[daily_activity['Id'] != 4057192912]
print(daily_activity.groupby("Id")["ActivityDate"].count())

Next if there are any days where no data was recorded by the FitBit other than calories, we need to remove them. It will shows calories because it will just automatically calculate the number of calories the user would burn just by being alive.

In [ ]:
daily_activity = daily_activity[daily_activity['TotalSteps'] != 0]
daily_activity.info()

Next, the daily_activity and daily_sleep tables have different names and formats for the date columns, so we need to rename and reformat these columns.

In [ ]:
daily_activity['ActivityDate'] = pd.to_datetime(daily_activity['ActivityDate'])
daily_sleep['ActivityDate'] = pd.to_datetime(daily_sleep['SleepDay'])

In [ ]:
daily_activity.info()
daily_activity.head()
daily_sleep.info()
daily_sleep.head()

In [ ]:
daily_activities = daily_activity.merge(daily_sleep, how = 'left', on = ['Id', 'ActivityDate'])
daily_activities.head()
#daily_activities.info()

There are multiple instances where the sleep data is missing for some users, so R automatically replaces these with null values.

We can use another quick query in BigQuery to figure out which uses are missing sleep entries.

SELECT Id, COUNT(ActivityDate) as total_days_sleep

FROM daily_sleep

GROUP BY Id

This query duplicated in R gives these results:

In [ ]:
print(daily_sleep.groupby('Id')['ActivityDate'].count())

Since we do not have a full 30 points of data for each user, any analysis done concerning this data will have to be taken with a grain of salt. We would also want to remove the users who have less than 10 entries. For now, we can keep these entries in the daily_activities table.

We will also create another column that combines the very active, fairly active, and lightly active minutes.

In [ ]:
daily_activities['total_active_mins'] = daily_activities['VeryActiveMinutes'] + daily_activities['FairlyActiveMinutes'] + daily_activities['LightlyActiveMinutes']
daily_activities.head()

Now we want to split up the date into days of the week to see if there is a difference in how the users behave on the weekdays vs weekends.

In [ ]:
from datetime import datetime

daily_activities['day_of_week'] = daily_activities['ActivityDate'].dt.strftime('%A')

Next we'll take a look at the heartrate data and try to clean it up. The FitBit only recorded the users heartrate at seemingly inconsistent intervals every few seconds. To make it a bit easier to read and analyze, we can aggregate the data to give the average heartrate each minute.

In [ ]:
heartrate_seconds.head()

In [ ]:
heartrate_seconds['date_time'] = pd.to_datetime(heartrate_seconds['Time'], format = "%m/%d/%Y %I:%M:%S %p")
#heartrate_seconds.info()

In [ ]:
print(heartrate_seconds)

In [ ]:
heartrate_minute = heartrate_seconds.groupby('Id').resample('1min', on = 'date_time', origin = '2016-04-12 07:21:00', offset = '07:21:00').Value.mean().reset_index()
print(heartrate_minute)

Now we have the heartrate in minutes for each user. We can compare the heartrate per minute with steps per minute and METs per minute to see if users had any spikes in heartrate that were not associated with any kind of exercise or rigorous movement.

There is a problem with the heartrate data because it only has data for 14 users and not for all 33. We can still try to analyze the data, but just like the sleep data, the results will have to be taken with a grain of salt.

In [ ]:
#retype to datetime instead of object
minute_steps['date_time'] = pd.to_datetime(minute_steps['ActivityMinute'], format = '%m/%d/%Y %I:%M:%S %p')
minute_mets['date_time'] = pd.to_datetime(minute_mets['ActivityMinute'], format = '%m/%d/%Y %I:%M:%S %p')
minute_calories['date_time'] = pd.to_datetime(minute_calories['ActivityMinute'], format = '%m/%d/%Y %I:%M:%S %p')

In [ ]:
#merge the minute tables together
minute_values = heartrate_minute.merge(minute_steps, how = 'left', on = ['Id', 'date_time'])
minute_values = minute_values.merge(minute_mets, how = 'left', on = ['Id', 'date_time'])
minute_values = minute_values.merge(minute_calories, how = 'left', on = ['Id', 'date_time'])
print(minute_values)

METs are measured based on the intensity of the exercise and the number of calories burned, according to the FitBit website and this FitBit help forum. The heartrate also apparently plays a role in the calculations. The exact way METs are calculated by the FitBits is not incredibly clear, but it does seem to be a combination of steps, heartrate, calories, and perceived intensity of the exercise.

Normally, 1 MET would be a person at rest doing no exercise, however, FitBits seem to record 1 MET as 10. All of the MET values they record are 10 times what they normally would be. This needs to be taken into account when analysing the data.

Now we will clean up the hourly_intensities and hourly_steps data.

In [ ]:
hourly_intensities['date_time'] = pd.to_datetime(hourly_intensities['ActivityHour'], format = "%m/%d/%Y %I:%M:%S %p")
hourly_intensities['time'] = hourly_intensities['date_time'].dt.time
hourly_intensities['day_of_week'] = hourly_intensities['date_time'].dt.strftime('%A')

hourly_steps['date_time'] = pd.to_datetime(hourly_steps['ActivityHour'], format = "%m/%d/%Y %I:%M:%S %p")
hourly_steps['time'] = hourly_steps['date_time'].dt.time
hourly_steps['day_of_week'] = hourly_steps['date_time'].dt.strftime('%A')

hourly_intensities.head()
hourly_steps.head()

# Analysis

First we will take a look at some averages for each day of the week in daily_activities and hourly_intensities.

In [ ]:
#let Python know to organize the days of the week in a specific order and not in alphabetical order
from pandas.api.types import CategoricalDtype

cat_day_order = CategoricalDtype(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], ordered = True)

daily_activities['day_of_week'] = daily_activities['day_of_week'].astype(cat_day_order)
daily_activities = daily_activities.sort_values('day_of_week')

hourly_intensities['day_of_week'] = hourly_intensities['day_of_week'].astype(cat_day_order)
hourly_intensities = hourly_intensities.sort_values('day_of_week')

hourly_steps['day_of_week'] = hourly_steps['day_of_week'].astype(cat_day_order)
hourly_steps = hourly_steps.sort_values('day_of_week')


ave_steps_days = daily_activities.groupby(['Id', 'day_of_week'])['TotalSteps'].mean().reset_index()
ave_calories_days = daily_activities.groupby(['Id', 'day_of_week'])['Calories'].mean().reset_index()
ave_sleep_days = daily_activities.groupby(['Id', 'day_of_week'])['TotalMinutesAsleep'].mean().reset_index()
ave_very_active_days = daily_activities.groupby(['Id', 'day_of_week'])['VeryActiveMinutes'].mean().reset_index()
ave_fairly_active_days = daily_activities.groupby(['Id', 'day_of_week'])['FairlyActiveMinutes'].mean().reset_index()
ave_lightly_active_days = daily_activities.groupby(['Id', 'day_of_week'])['LightlyActiveMinutes'].mean().reset_index()
ave_total_active_days = daily_activities.groupby(['Id', 'day_of_week'])['total_active_mins'].mean().reset_index()

mean_intensities_hours = hourly_intensities.groupby(['time', 'day_of_week'])['TotalIntensity'].mean().reset_index()
mean_steps_hours = hourly_steps.groupby(['time', 'day_of_week'])['StepTotal'].mean().reset_index()


If we want to include an analysis of the sedentary mintues, there is something we need to consider. For multiple entries, the number of sedentary minutes appears to include the time the user spent asleep, giving a value of around 1200 minutes. Since there are only 1440 minutes in a day, it is reasonable to assess that at least some of those 1200 minutes were spent sleeping. If we want to do an analysis of the sedentary minutes, we need to decide if those entries should be included and if so, then for the other entries, do we add the minutes spent asleep to the sedentary minutes as well.

In [ ]:
(daily_activities[daily_activities['SedentaryMinutes'] > 1000]).shape

There's over 400 entries where at least 1000 sedentary minutes were logged by the FitBit. We will hold off doing any further analysis on the sedentary minutes as we can fairly safely assume that any time not spent active or sleeping was likely spent sedentary.

Next, we will check to see how many times any of the users logged an activity distance since I noticed that almost all of the entries in that column are a zero.

In [ ]:
cols = ['Id', 'ActivityDate', 'day_of_week', 'LoggedActivitiesDistance']
logged_activity = daily_activities.loc[daily_activities['LoggedActivitiesDistance'] != 0, cols]

print(logged_activity)

We can see that only 4 uses ever logged an activity distance, and only 2 of them logged more than just a few times. This is not enough data to draw any reasonably accurate conclusions about FitBit users that could help Bellabeat beyond recognizing that users tend to not manually log activities.

Now we'll take a look at the minute_values table. We know that generally we can expect the user's heartrate to increase while they are active. We can take a look to see if the recorded heartrates increase when the number of steps and METs increasea as well.

In [ ]:
elevated_minute_values = minute_values[minute_values['Value'] >= 100]

raw_daily_activity = pd.read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
zero_recorded = raw_daily_activity.loc[raw_daily_activity['SedentaryMinutes'] == 1440, 'Calories']
print(zero_recorded.mean())

In [ ]:
print(zero_recorded)

We use 100 BPM as the cutoff for an elevated heartrate because normal resting heart rate for a healthy adult should be between 60 and 100 BPM according to [Harvard Medical](https://www.health.harvard.edu/heart-health/what-your-heart-rate-is-telling-you).

The entries in the daily_activity where nothing is recorded except for sedentary minutes and the number of calories burned, an average of 1792 calories were apparently burned. This means the FitBit uses that number as a baseline for the bare minimum number of calories a user will burn just by being alive for 24 hours, which works out to about 1.24 calories on average per minute.

In [ ]:
#look at the averages for each of the columns to see if they roughly line up with what we would expect
print(elevated_minute_values['Value'].mean())
print(elevated_minute_values['Steps'].mean())
print(elevated_minute_values['METs'].mean())
print(elevated_minute_values['Calories'].mean())

The average number of METs and calories burned while the user's heartrate was above 100 roughly line up with each other. 1.24 times 5 is 6.2 while 10 times 5 is 50.

Now we want to see if there are instances where the users have an elevated heartrate but no increase in METs, calories, or steps.

In [ ]:
irregular_values = elevated_minute_values.loc[elevated_minute_values['METs'] < 30,: ]
print(irregular_values)

A quick scan through this new table shows that there are several instances where there is a span of several minutes, sometimes roughly half an hour, where the user's heartrate is above 100, but the number of METs is below 30. 30 is used as a baseline because the FitBit will only record an active minute if the MET is above 30 for 10 minutes. These instances of low METs but increased heartrate are also accompanied by a low number of steps and calories. It's unlikely that these instances are caused by exercises that don't register steps as the FitBit website indicates that if the model records heartrate, it tends to be able to more accurately record that kind of exercise as well. The most prominent thing that comes to mind that could cause an increased heartrate with little to no activity is stress.

# Visualizing the Data and Drawing Conclusions

In [ ]:
#import matplotlib.pyplot as plt
import seaborn as sns

#graph the number of steps the uers take for each day of the week
step_means = ave_steps_days.groupby(['day_of_week'])['TotalSteps'].mean().round()
vert_offset = ave_steps_days['TotalSteps'].mean() * 0.05
step_boxplot = sns.boxplot(x = ave_steps_days['day_of_week'], y = ave_steps_days['TotalSteps'], showmeans = True)

for xtick in step_boxplot.get_xticks():
    step_boxplot.text(xtick, step_means[xtick] + vert_offset, step_means[xtick], horizontalalignment = 'center', size = 'x-small', color = 'w', weight = 'semibold')

The average number of steps, represented by the labeled dot in the middle of each box, varied from day to day with the lowest average on Sunday. On average, users walked around 7,000 - 8,000 steps each day. The [CDC](https://www.cdc.gov/diabetes/prevention/pdf/postcurriculum_session8.pdf) recommends that an adult should walk 10,000 steps a day meaning that these users, on average, walked fewer than the recommend number of steps.

The boxplots also show that the number of steps varied much more towards a higher step count on Saturday. This could be from the users inconsistently trying to be more active on that day. However, on Wednesday and Thursday, we can see that the variation in steps was the smallest. It's likely that users in general tend to not try to take more steps in the middle of the week but also do not end up taking any fewer than usual.

Sunday has a low average but the second largest variation. This could be similar to Saturday where users inconsistently try to be more active on the weekends or inconsistently resting or remaining sedentary on Sundays.

In [ ]:
#make a graph showing the average number of steps per hour
from plotnine import *
plot1 = ggplot(mean_steps_hours) + aes(x = 'time', y = 'StepTotal', fill = 'day_of_week') + geom_col() + theme(axis_text_x = element_text(angle = 90))
plot2 = ggplot(mean_steps_hours) + aes(x = 'time', y = 'StepTotal') + geom_col() + facet_wrap(['day_of_week']) + theme(axis_text_x = element_text(angle = 90))

print(plot1)
print(plot2)

We can see that on overall, users tended to walk a bit more around 6-7 p.m. If we look at the breakdown per day of the week, We can see that for Monday,Tuesday, Wednesday, and Friday, users appear to also consistently follow this trend of walking more when most people get off work. Wednesday has hte largest spike with a difference of around 300 steps.

Tuesday, however, also has a spike from 11 a.m. to 1 p.m. This might be from users making an effort to take more steps around lunch time or walking somewhere to buty their lunch (taco Tuesday?).

Thursday does not have a significant spike in steps at any point throughout the day. Users appear to take slightly more around lunch time and after work, less than 200 more, but the increase is not has sharp as the other days where users walked well over 200 more steps.

Saturday shows a very noticeable spike in steps around lunch time, almost 400 steps more, with a much smaller spike at dinner time. Sunday, however, more closely resembles Thursday as it does not have significant spikes like the other days.

If we take a look at users' total daily active minutes, we can potentially get a better idea if these extra steps correlate to extra exercise.

In [ ]:
#activity averages
activity_means = ave_total_active_days.groupby(['day_of_week'])['total_active_mins'].mean().round()
vert_offset = ave_total_active_days['total_active_mins'].mean() * 0.05
active_boxplot = sns.boxplot(x = ave_total_active_days['day_of_week'], y = ave_total_active_days['total_active_mins'], showmeans = True)

for xtick in active_boxplot.get_xticks():
    active_boxplot.text(xtick, activity_means[xtick] + vert_offset, activity_means[xtick], horizontalalignment = 'center', size = 'x-small', color = 'w', weight = 'semibold')

This boxplot and the one for steps are fairly similar. The distribution of the medians is more consistent for this graph than the steps one, but the overall trend of the averages match up nicely between the two. We can see that the FitBits use steps are part of the way to calculate active minutes in general. This makes sense. Saturday, just like in the steps boxplot, has the highest average number of active minutes. This supports the idea that users try to be more active on Saturday.

In [ ]:
plot3 = ggplot(mean_intensities_hours) + aes(x = 'time', y = 'TotalIntensity', fill = 'day_of_week') + geom_col() + theme(axis_text_x = element_text(angle = 90))
plot4 = ggplot(mean_intensities_hours) + aes(x = 'time', y = 'TotalIntensity') + geom_col() + facet_wrap(['day_of_week']) + theme(axis_text_x = element_text(angle = 90))

print(plot3)
print(plot4)

The histograms for hourly intensities also match up nicely with the histograms for hourly steps. Since intensities, active minutes, and steps are all correlated to each other in the FitBit's calculations, this make sense and supports the observations made from the steps graphs.

In [ ]:
#comparing each active minute type with sedentary minutes
#sedentary minutes includes time spent sleeping
mean_min_types = ave_lightly_active_days.merge(ave_fairly_active_days, how = 'left', on = ['Id', 'day_of_week'])
mean_min_types = mean_min_types.merge(ave_very_active_days, how = 'left', on = ['Id', 'day_of_week'])
mean_min_types['sedentary'] = 1440 - ave_total_active_days['total_active_mins']

mean_all_days_types = mean_min_types[['LightlyActiveMinutes', 'FairlyActiveMinutes', 'VeryActiveMinutes', 'sedentary']].mean().reset_index()
mean_all_days_types.rename(columns = {"index": "type", list(mean_all_days_types)[1]: "Minutes"}, inplace = True)
mean_all_days_types.head()

In [ ]:
import matplotlib.pyplot as plt
import math

type_labels = mean_all_days_types['type']
type_totals = mean_all_days_types['Minutes'].round()
#type_totals = pd.to_numeric(mean_all_days_types['Minutes'].round(), downcast = 'integer')

def minute_values(val):
    a = np.round(val/100.*type_totals.sum(), 0)
    #a = pd.to_numeric(a, downcast = 'integer')
    return math.trunc(a)

plot5 = plt.pie(type_totals, labels = type_labels, autopct = minute_values, pctdistance = 0.9, rotatelabels = True, radius = 1.5)



In [ ]:
mean_min_types2 = mean_min_types[['day_of_week', 'LightlyActiveMinutes', 'FairlyActiveMinutes', 'VeryActiveMinutes', 'sedentary']]
mean_min_types2 = mean_min_types2.groupby('day_of_week')['LightlyActiveMinutes', 'FairlyActiveMinutes', 'VeryActiveMinutes', 'sedentary'].mean().reset_index()
mean_min_types2_l = pd.melt(mean_min_types2, id_vars = "day_of_week", value_vars = ['LightlyActiveMinutes', 'FairlyActiveMinutes', 'VeryActiveMinutes', 'sedentary'])

mean_min_types2_long = mean_min_types2_l.pivot_table(index = ["variable"], columns = 'day_of_week', values = 'value')

#create a pie chart for each day of the week starting with Sunday going from left to right
sub_pies = mean_min_types2_long.plot.pie(subplots = True, legend = False, figsize = (150, 200), fontsize = 70, rotatelabels = True, labeldistance = 0.5)


The [CDC](https://www.cdc.gov/physicalactivity/basics/adults/index.htm) recommends 150 minutes of moderate-intensity aerobic activity each week or 30 minutes a day, 5 days a week. We can see that, on average, the users had 15 minutes of fairly active exercise and 22 minutes of very active exercise a day. If we consider both of these to be at least "moderate-intensity" then the users are exercising for an average of 37 minutes each day which exceeds the recommended amount.

In [ ]:
#graphing sleep averages
sleep_means = ave_sleep_days.groupby('day_of_week')['TotalMinutesAsleep'].mean().round()
vert_offset2 = ave_sleep_days['TotalMinutesAsleep'].mean() * 0.05
sleep_boxplot = sns.boxplot(x = ave_sleep_days['day_of_week'], y = ave_sleep_days['TotalMinutesAsleep'], showmeans = True)

for xtick in sleep_boxplot.get_xticks():
    sleep_boxplot.text(xtick, sleep_means[xtick] + vert_offset2, sleep_means[xtick], horizontalalignment = 'center', size = 'x-small', color = 'w', weight = 'semibold')

In the graph above, the amount of sleep shown for each day is for the night leading into that day. So, for example, on Sunday, the average amount of sleep the users got was 425 minutes meaning that they slept for 425 minutes on the night between Saturday and Sunday.

Users tended to get an average of 6 - 7 hours of sleep each night, which, depending on the person, may or may not be adequate. This could be something that BellaBeat could look into.

Additionally, there is a very wide range of sleep minutes on the night between Friday and Saturday. Users likely either stayed up very late Friday night or slept in on Saturday morning resulting in such a large distribution. A similar, but to a lesser degree, explanation could apply to the night between Saturday and Sunday, but it appears that more people consistently opted to either sleep in or go to bed early as the median line is quite high on the boxplot.



# Conclusion and Recommendations

### Exercise

FitBit users overall do appear to exercise the recommended amount per week but do not reach the recommended 10,000 steps per day. Bellabeat could have the Bellabeat app notify users when they've reached 150 minutes a week of exercise and when they reach 10,000 steps. If the week is almost over and the user still has not reached these goals or if they are well below the average number of minutes and steps they would normally be at by that day, the app could send a notification either on the user's phone or their fitness tracker. A simple reminder could help users stay on track with their fitness goals.

Also, although calories were not closely analyzed in this report, Bellabeat could also consider adding a calorie tracker to the app. Users could log the foods they consume throughout the day, and the fitness tracker would tell the app how many calories the user burned that day. The app could then tell the user if they had a calorie surplus or deficit that day and tell users how to adjust their calorie intake to match their fitness goals.

### Manual Logging

The data also suggests that users do not consistently manually log data. Only 8 users logged their weight, and of those 8, only one user had data for the whole month. Additionally, only 4 users every logged the distance for an activity. Bellabeat could try to reminder users to log information like their weight every week or so. It would be difficult for Bellabeat to create something that would automatically measure the user's weight without them stepping on a scale every week, so a simple reminder would be a simple compromise.

### Heartrate

Most fitness trackers today tend to measure the user's heartrate regularly. Bellabeat could have the app track the user's average resting heartrate as well as the heartrate while exercising. It could then notify users if their resting heartrate has gone down or up which can be a good indicator of whether their overal vascular health has improved or worsened. It would also be useful for the app to be able to take the informaion from the tracker to tell if the user is exercising or not. If the user is not exercising but their heartrate is elevated, the app could make a note of this and notify the user if it stays elevated for a certain amount of time. Elevated heartrate without exercise can be an idicator of stress or heart problems. If the user regularly has a spike in their heartrate at a certain time of day when they are sedentary, then it could be an indicator that something is happening in their life at that specific time that causes them a lot of stress or excitement. The user could then make an informed decision on ways they could reduce their stress.

### Menstrual Cycle

The data does not indicate if the users are female or male or the age of each user. Since Bellabeat focuses on women and their health, the app could act as a menstrual cycle tracker for women who have not reached menopause yet. The app could then analyze how the user's menstrual cycle could be affecting their activity levels or the amount of sleep they get. This information could help users adjust their habits to accomodate for these changes.